In [1]:
%%javascript
var width = window.innerWidth || document.documentElement.clientWidth || document.body.clientWidth;
var height = window.innerHeight || document.documentElement.clientHeight || document.body.clientHeight;

IPython.notebook.kernel.execute("windowSize = (" + width + "," + height + ")");
// suitable for small screens
nbpresent.mode.tree.set(
    ["app", "theme-manager", "themes", "my-theme"], 
    {
    palette: {
        "blue": { id: "blue", rgb: [0, 153, 204] },
        "black": { id: "black", rgb: [0, 0, 0] },
        "white": { id: "white", rgb: [255, 255, 255] },
        "red": { id: "red", rgb: [240, 32, 32] },
        "gray": { id: "gray", rgb: [128, 128, 128] },
    },
    backgrounds: {
        "my-background": {
            "background-color": "white"
        }
    },
    "text-base": {
        "font-family": "Georgia",
        "font-size": 2.5
    },
    rules: {
        h1: {
            "font-size": 5.5,
            color: "blue",
            "text-align": "center"
        },
        h2: {
            "font-size": 3,
            color: "blue",
            "text-align": "center"
        },
        h3: {
            "font-size": 3,
            color: "black",
        },
        "ul li": {
            "font-size": 2.5,
            color: "black"
        },
        "ul li ul li": {
            "font-size": 2.0,
            color: "black"
        },
        "code": {
            "font-size": 1.6,
        },
        "pre": {
            "font-size": 1.6,
        }
    }
});


<IPython.core.display.Javascript object>

# Combinatorial Pattern Matching

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21CombinatorialPillowTalk.jpg" width="700px">

<p style="text-align: right; clear: right;">1</p>

# A Recurring Problem

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21FindPattern.jpg" width="400px" style="float: right; margin: 0px 80px 0px 80px;">

* Finding patterns within sequences<br><br>
* Variants on this idea
   - Finding repeated motifs amoungst a set of strings 
   - What are the most frequent k-mers
   - How many time does a specific k-mer appear<br><br>
* Fundamental problem: *Pattern Matching*
   - Find all positions of a particular substring in given sequence?
<br><br><br><br><br><br>
<p style="text-align: right; clear: right;">2</p>

# Pattern Matching 

* <u>**Goal:**</u> Find all occurrences of a pattern in a text

* <u>**Input:**</u> Pattern $p = p_1, p_2, … p_n$ and text $t = t_1, t_2, … t_m$

* <u>**Output:**</u> All positions 1 < i < (m – n + 1) such that the *n*-letter substring of t starting at i matches p

In [11]:
def bruteForcePatternMatching(p, t):
    locations = []
    for i in xrange(0, len(t)-len(p)+1):
        if t[i:i+len(p)] == p:
            locations.append(i)
    return locations

print bruteForcePatternMatching("ssi", "imissmissmississippi")

[11, 14]


<p style="text-align: right; clear: right;">3</p>

# Pattern Matching Performance

* Performance:
     - *m* - length of the text *t*
     - *n* - the length of the pattern *p*
     - Search Loop - executed *O(m)* times
     - Comparison - *O(n)* symbols compared 
     - Total cost - *O(mn)* per pattern

* In practice, most comparisons terminate early

* Worst-case:
     - <code>p = "AAAT"</code>
     - <code>t = "AAAAAAAAAAAAAAAAAAAAAAAT"</code>

<p style="text-align: right; clear: right;">4</p>

# We can do better!

If we preprocess our pattern we can search more effciently (*O(n)*)

Example:
<pre style="font-size: 80%">    
            imissmissmississippi
         1. s                      
         2.  s
         3.   s
         4.    SSi
         5.       s
         6.        SSi
         7.           s
         8.            SSI             - match at 11
         9.               SSI          - match at 14
         10.                 s
         11.                  s
         12.                   s
</pre>

* At steps 4  and 6 after finding the mismatch $i \ne m$ we can skip over all positions tested because we know that the suffix *"sm"* is not a prefix of our pattern *"ssi"*
* Even works for our worst-case example "AAAAT" in "AAAAAAAAAAAAAAT" by recognizing the shared prefixes ("AAA" in "AAAA").
* How about finding multiple patterns $[p_1, p_2, ..., p_3]$ in $t$

<p style="text-align: right; clear: right;">5</p>

# Keyword Trees 

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21KeywordTree.png" width="300px" style="float: right; margin: 0px 80px 0px 80px;">

* We can preprocess the set of strings we are seeking to minimize the number of comparisons

* **Idea:** Combine patterns that share prefixes, to *share* those comparisons
    - Stores a set of keywords in a rooted labeled tree
    - Each edge labeled with a letter from an alphabet
    - All edges leaving a given vertex have distinct labels
    - Leaf vertices are indicated
    - Every keyword stored can be spelled on a path from root to some leaf vertex
    - Searches are performed by “threading” the target pattern through the tree 

* A tree is a special graph as discussed previously
    - one connected component
    - *N* nodes
    - *N-1* edges
    - No loops
    - Exactly one path from any. 

* A ***Trie*** is a tree that is related to a sequence.
    - Generally, there is a 1-to-1 correspondence between either nodes or edges of the *trie* and a symbol of the sequence 
<p style="text-align: right; clear: right;">6</p>

# Prefix *Trie* Match

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21KeywordTree.png" width="300px" style="float: right; margin: 0px 80px 0px 80px;">
* <u>**Input:**</u> A text *t* and a trie *P* of patterns
* <u>**Output:**</u> True if *t* leads to a leaf in *P*; False otherwise

What is output for:
   * *apple*
   * *band*
   * *april*
   
Performance:
   * $O(m)$ - the length of the text, *t*
   * Independent of how many strings are in the Keyword Trie

<p style="text-align: right; clear: right;">7</p>

In [42]:
def path(string, parent):
    if (len(string) > 0):
        if (string[0] in parent):
            child = parent[string[0]]
        else:
            child = {}
            parent[string[0]] = child
        path(string[1:], child)
    else:
        parent['$'] = True

class PrefixTrie:
    def __init__(self):
        """ Tree is a dictionary of the children at each node"""
        self.root = {}
    def add(self, string):
        """ Add a path from the Trie's root"""
        path(string, self.root)
    def match(self, string):
        """ Check if there is a path from the root to a '$' """
        parent = self.root
        for c in string:
            if c not in parent:
                break
            parent = parent[c]
        return '$' in parent


T = PrefixTrie()
T.add("apple")
T.add("banana")
T.add("apricot")
T.add("bandana")
T.add("orange")
print T.root
print map(T.match, ['apple', 'banana', 'apricot', 'orange', 'band', 'april', 'bananapple'])


{'a': {'p': {'p': {'l': {'e': {'$': True}}}, 'r': {'i': {'c': {'o': {'t': {'$': True}}}}}}}, 'b': {'a': {'n': {'a': {'n': {'a': {'$': True}}}, 'd': {'a': {'n': {'a': {'$': True}}}}}}}, 'o': {'r': {'a': {'n': {'g': {'e': {'$': True}}}}}}}
[True, True, True, True, False, False, True]


# Multiple Pattern Matching

* *t* - the text to search through
* *P* - the trie of patterns to search for

``` python
def multiplePatternMatching(t, P):
    locations = []
    for i in xrange(0, len(t)):
        if PrefixTrieMatch(t[i:], P):
            locations.append(i)
    return locations
```

<p style="text-align: right; clear: right;">8</p>

# Multiple Pattern Matching Example

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21KeywordTree.png" width="320px" style="float: right; margin: 0px 80px 0px 80px;">

``` python
multiplePatternMatching(“bananapple”, P):
 0:  PrefixTrieMatching(“bananapple”, P) = True
 1:  PrefixTrieMatching(“ananapple”, P) = False
 2:  PrefixTrieMatching(“nanapple”, P) = False
 3:  PrefixTrieMatching(“anapple”, P) = False
 4:  PrefixTrieMatching(“napple”, P) = False
 5:  PrefixTrieMatching(“apple”, P) = True
 6:  PrefixTrieMatching(“pple”, P) = False
 7:  PrefixTrieMatching(“ple”, P) = False
 8:  PrefixTrieMatching(“le”, P) = False
 9:  PrefixTrieMatching(“e”, P) = False

locations = [0, 5]
```
<p style="text-align: right; clear: right;">9</p>

# Improvements

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21Multistring.png" width="300px" style="float: right; margin: 0px 80px 0px 80px;">

* Based on our previous speed-up
* We can add failure edges to our Trie
* *Aho-Corasick* Algorithm

<code style="font-size: 180%; line-height: 1.2;">
    bapple
    <span style="color: blue;">ba</span><span style="color: red;">p</span>
     <span style="color: gray;">a</span><span style="color: blue;">pple</span>
</code>

<p style="text-align: right; clear: right;">10</p>

# Multiple Pattern Matching Performance

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21KeywordTree.png" width="320px" style="float: right; margin: 0px 80px 0px 80px;">
* m - len(t)
* d - max depth of P (longest pattern in P)
* O(md) to find all patterns
* Can be decreased further to O(m) using Aho-Corasick Algorithm (see pg 353)
* Memory issues
    - Tries require a lot of memory
    - Practical implementation is challenging
    - Genomic reads - millions to billions of

* Patterns typically of length > 100

<p style="text-align: right; clear: right;">11</p>

# Another Twist

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21KeywordSuffix.png" width="320px" style="float: right; margin: 0px 80px 0px 80px;">

<ul style="font-size: 120%;">
<li>What if our list of keywords were simply all suffixes of a given string</li>
<pre style="font-size: 120%;">
      Example: ACATG
                CATG
                 ATG
                  TG
                   G
</pre>
<li>The resulting keyword tree:</li>
<li>A <strong>Suffix Trie</strong></li>
</ul>

<p style="text-align: right; clear: right;">12</p>

# Suffix Tree

### A *compressed* Suffix Trie

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21KeywordSuffix.png" width="300px" style="float: left; margin: 0px 80px 0px 40px;">
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21SuffixTree.png" width="300px" style="float: right; margin: 0px 40px 0px 80px;">

<ul>
<li style="font-size: 120%">Combines nodes with in and out degree 1</li>
<li style="font-size: 120%">Edges are text substrings</li>
<li style="font-size: 120%">All internal nodes have at least 3 edges</li>
<li style="font-size: 120%">All leaf nodes are labeled with an index</li>
</ul>
<br><br><br><br><br>
<p style="text-align: right; clear: left;">13</p>

# Uses for Suffix Trees

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21SuffixTree.png" width="400px" style="float: right; margin: 0px 80px 0px 40px;">
* Suffix trees hold all suffixes of a text, T
    - i.e., ATCGC: ATCGC, TCGC, CGC, GC, C
* Can be built in $O(m)$ time for text of length $m$
* To find any pattern P in a text:
    - Build suffix tree for text, $O(m)$, $m = |T|$
    - Thread the pattern through the suffix tree
    - Can find pattern in $O(n)$ time! ($n = |P|$)
* $O(|T| + |P|)$ time for "Pattern Matching Problem"<br>(better than Naïve O(|P||T|)
* Build suffix tree and lookup pattern
* Multiple Pattern Matching in $O(|T| + k|P|)$

<p style="text-align: right; clear: left;">14</p>

# Suffix Tree Overhead 

* Input: text of length m
* Computation
  - O(m) to compute a suffix tree
  - Does not require building the suffix trie first
* Memory
  - O(m) - nodes are stored as offsets and lengths

* Huge hidden constant, best implementations

* Requires about 20*m bytes

* 3 GB human genome = 60 GB RAM

<p style="text-align: right; clear: left;">15</p>

# Suffix Tree Examples

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21STExample.png" width="600px" style="float: right; margin: 0px 80px 0px 80px;">
* What is the string represented in the suffix tree?
* What letter occurs most frequently?
* How many times doaes "ATG" appear, and where?
* How long is the longest repeated k-mer?

<p style="text-align: right; clear: right;">16</p>

# Suffix Trees: Theory vs. Practice

* In theory, suffix trees are extremely powerful for making a variety of queries concerning a sequence
  - What is the shortest unique substring?
  - How many times does a given string appear in a text?
* Despite the existence of linear-time construction algorithms, and O(m) search times, suffix trees are still rarely used for genome-scale searching
* Large storage overhead

<p style="text-align: right; clear: right;">17</p>

# Substring Searching

* Is there some other data structure to gain efficent access to all of the suffixes of a given string with less overhead than a suffix tree?
* Some things we know
    - Searching an unordered list of items with length *n* generally requires *O(n)* steps
    - However, if we sort our items first, then we can search using *O(log(n))* steps
    - Thus, if we plan to do frequent searchs there is some advantage to performing a sort first and amortizing its cost over many searchs
* For strings *suffixes* are interesting *items*. Why?

        Suffixes: panamabananas        Sorted Suffixes:  abananas
                  anamabananas                           amabananas
                  namabananas                            anamabananas
                  amabananas                             ananas
                  mabananas                              anas
                  abananas                               as
                  bananas                                bananas
                  ananas                                 mabananas
                  nanas                                  namabananas
                  anas                                   nanas
                  nas                                    nas
                  as                                     panamabananas
                  s                                      s
<p style="text-align: right; clear: right;">18</p>

# Questions you can ask

Is there any use for a list of sorted suffixes?

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L21Questions.jpg" style="float: left; margin: 0px 0px 0px 20px;">

          Sorted Suffixes:  abananas
                            amabananas
                            anamabananas
                            ananas
                            anas
                            as
                            bananas
                            mabananas
                            namabananas
                            nanas
                            nas
                            panamabananas
                            s

<br style="clear: left;">
* Does the substring "nana" appear in the orginal string? How?
* How many times does "ana" appear in the string?
* What is the most/least frequent letter in the orginal string?
* What is the most frequent two-letter substring in the orginal string?

<p style="text-align: right; clear: right;">19</p>

# Properties of a Naive sorted suffix implementation

* Size of the sorted list if the given text has a length of *m*?  $O(m^2)$
* Cost of the sort? $O(m^2 log(m))$
* Not practical for big *m*

* There are many ways to sort
   - What is an *in place* sort?
   - What is a *stable* sort?
   - What is an *arg sort*?

<p style="text-align: right; clear: right;">20</p>

# Arg Sorting

Consider the list: <pre>[7,2,4,3,1,5,0,6]</pre>

When sorted it is simply: <pre>[0,1,2,3,4,5,6,7]</pre>

Its arg sort is: <pre>[6, 4, 1, 3, 2, 5, 7, 0]</pre>

* The *i<sup>th</sup>* element in the arg sort is the *index* of the *i<sup>th</sup>* element from the orginal list when sorted.
* Thus, <code>[A[i] for i in argsort(A)] == sorted[A]</code>

<p style="text-align: right; clear: right;">21</p>

# Code for Arg Sorting

In [41]:
def argsort(input):
    return sorted(range(len(input)), cmp=lambda i,j: 1 if input[i] >= input[j] else -1)

A = [7,2,4,3,1,5,0,6]
print argsort(A)
print [A[i] for i in argsort(A)]

print
B = ["TAGACAT", "AGACAT", "GACAT", "ACAT", "CAT", "AT", "T"]
print argsort(B)
print [B[i] for i in argsort(B)]

[6, 4, 1, 3, 2, 5, 7, 0]
[0, 1, 2, 3, 4, 5, 6, 7]

[3, 1, 5, 4, 2, 6, 0]
['ACAT', 'AGACAT', 'AT', 'CAT', 'GACAT', 'T', 'TAGACAT']


<p style="text-align: right; clear: right;">22</p>

# Next Time

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/suffixCartoon.gif" width="400px" style="float: right; margin: 0px 80px 0px 80px;">
* We'll see how arg sorting can be used to simplify representing our sorted list of suffixes
* Suffix arrays
* Burrows-Wheeler Transforms
* Applications in sequence alignment

<p style="text-align: right; clear: right;">23</p>